In [0]:
!cat /etc/os-release
!nvcc --version

In [19]:
!curl -o trt.deb https://at.ispras.ru/owncloud/index.php/s/pOGMehXNaRuIj73/download
!dpkg -i trt.deb
!find /var/nv*
!apt-key add /var/nv-tensorrt-repo-cuda10.1-trt6.0.1.5-ga-20190913/7fa2af80.pub
!apt-get update
!VER=6.0.1-1+cuda10.1 ; apt-get install libnvinfer6=$VER libnvinfer-plugin6=$VER libnvparsers6=$VER libnvonnxparsers6=$VER libnvinfer-bin=$VER libnvinfer-dev=$VER libnvinfer-plugin-dev=$VER libnvparsers-dev=$VER libnvonnxparsers-dev=$VER libnvinfer-samples=$VER libnvinfer-doc=$VER python3-libnvinfer=$VER python3-libnvinfer-dev=$VER
!apt-get install tensorrt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  900M  100  900M    0     0  63.0M      0  0:00:14  0:00:14 --:--:-- 62.9M
(Reading database ... 146977 files and directories currently installed.)
Preparing to unpack trt.deb ...
Unpacking nv-tensorrt-repo-ubuntu1804-cuda10.1-trt6.0.1.5-ga-20190913 (1-1) over (1-1) ...
Setting up nv-tensorrt-repo-ubuntu1804-cuda10.1-trt6.0.1.5-ga-20190913 (1-1) ...
/var/nv-tensorrt-repo-cuda10.1-trt6.0.1.5-ga-20190913
/var/nv-tensorrt-repo-cuda10.1-trt6.0.1.5-ga-20190913/libnvinfer-samples_6.0.1-1+cuda10.1_all.deb
/var/nv-tensorrt-repo-cuda10.1-trt6.0.1.5-ga-20190913/python-libnvinfer-dev_6.0.1-1+cuda10.1_amd64.deb
/var/nv-tensorrt-repo-cuda10.1-trt6.0.1.5-ga-20190913/libcudnn7-dev_7.6.3.30-1+cuda10.1_amd64.deb
/var/nv-tensorrt-repo-cuda10.1-trt6.0.1.5-ga-20190913/7fa2af80.pub
/var/nv-tensorrt-repo-cuda10.1-trt6.0.1.5-ga-20190913/uff-converte

In [0]:
# Runtime -> Restart runtime...
import os
def restart_runtime():
    os.kill(os.getpid(), 9)
restart_runtime()

In [1]:
import tensorrt
tensorrt.__version__
!pip install git+https://github.com/NVIDIA-AI-IOT/torch2trt.git

  Cloning https://github.com/NVIDIA-AI-IOT/torch2trt.git to /tmp/pip-req-build-xojsuj9c
  Running command git clone -q https://github.com/NVIDIA-AI-IOT/torch2trt.git /tmp/pip-req-build-xojsuj9c
  Created wheel for torch2trt: filename=torch2trt-0.0.3-cp36-none-any.whl size=43738 sha256=15b94d4784e8e34b185e55b311ea80f01fda029e58c3a4c9f007b93fcc31513e
  Stored in directory: /tmp/pip-ephem-wheel-cache-dd571xc3/wheels/75/8e/d1/b744a314f8312c6b2968d111c479cbd110d94bff1cbac7401d
Successfully built torch2trt


In [0]:
from torch2trt import torch2trt

## Задание 2

Состоит из **обязательной** и **бонусной** частей.

Обязательная часть оценивается в **50 баллов** и выполняется до **16 декабря 09:00**.

Бонусную часть можно делать, пока не придет необходимость получения оценки/зачета.

Обязательная часть заключается в fine-tuning несложной нейросети (ResNet-18) на UIUC Sports Event Dataset (http://vision.stanford.edu/lijiali/event_dataset) и ее последующем ускорении с помощью фреймворка **NVIDIA TensorRT** (https://developer.nvidia.com/tensorrt).

In [0]:
# !pwd

/content


In [0]:
import torch
import torchvision
from torchvision import transforms, models
from torchvision.datasets import ImageFolder
import copy as cp

import numpy as np
import random
import timeit

from tqdm import tqdm_notebook as tqdm

**(5 баллов)** Скачайте датасет, распакуйте его в директорию `./event_img/`. В ней должны оказаться 8 директорий, соответствующих классам картинок. Загрузите датасет в torch и разбейте случайным образом на train и val.

In [9]:
!wget http://vision.stanford.edu/lijiali/event_dataset/event_dataset.rar
!rm -rf data
!unrar x event_dataset.rar
!mv event_img data

--2019-12-15 21:30:57--  http://vision.stanford.edu/lijiali/event_dataset/event_dataset.rar
Resolving vision.stanford.edu (vision.stanford.edu)... 171.64.68.10
Connecting to vision.stanford.edu (vision.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 495597858 (473M) [text/plain]
Saving to: ‘event_dataset.rar’

event_dataset.rar   100%[===================>] 472.64M  19.6MB/s    in 27s     

2019-12-15 21:31:24 (17.4 MB/s) - ‘event_dataset.rar’ saved [495597858/495597858]


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from event_dataset.rar

Creating    event_img                                                 OK
Creating    event_img/badminton                                       OK
Extracting  event_img/badminton/Easy_Close_badminton_107.jpg               0%  OK 
Extracting  event_img/badminton/Easy_Close_badminton_130.jpg               0%  OK 
Extracting  event_img/badminton/Easy_C

In [0]:
def fix_seed(seed=23):
    np.random.seed(seed)
    random.seed(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [11]:
fix_seed()

event_dataset = ImageFolder('/content/data')
assert isinstance(event_dataset, torch.utils.data.Dataset)

dataset_length = len(event_dataset)
print(f'Number of images: {dataset_length}')

Number of images: 1579


In [0]:
val_size = .4
val_length = int(dataset_length * val_size)

train_data_raw, val_data_raw = torch.utils.data.random_split(event_dataset, [dataset_length - val_length, val_length])

assert isinstance(train_data_raw, torch.utils.data.Dataset)
assert isinstance(val_data_raw, torch.utils.data.Dataset)

**(10 баллов)** Нам нужны разные преобразования (transforms) для train и val. Напишите класс `ApplyTransform`, объект которого — тот же датасет, что подается в конструкторе, но с примененными преобразованиями.

К `train_data_raw` нужно применить изменение размера до 256px (`min(height, width)`), извлечение региона 256x256 в центре, выбор в этом регионе случайного квадрата 224x224.

К `test_data_raw` нужно применить изменение размера до 224px и извлечение региона 224x224 в центре.

In [0]:
class ApplyTransform(torch.utils.data.Dataset):        
    def __init__(self, dataset, transform=None, target_transform=None):
        self.dataset = dataset
        self.transform = transform
    
    def __getitem__(self, index):
        return (self.transform(self.dataset[index][0]) if self.transform else transforms.ToTensor()(self.dataset[index][0]), self.dataset[index][1])

    def __len__(self):
        return len(self.dataset)

In [0]:
imagenet_normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

train_transform = transforms.Compose([
    transforms.Resize(256, interpolation=2),
    transforms.CenterCrop(256),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    imagenet_normalize
])

val_transform = transforms.Compose([
    transforms.Resize(224, interpolation=2),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    imagenet_normalize
])

train_data = ApplyTransform(train_data_raw, train_transform)
val_data = ApplyTransform(val_data_raw, val_transform)

In [0]:
batch_size = 32
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=4)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, num_workers=4)

**(5 баллов)** Загрузите предобученную на ImageNet модель ResNet-18, адаптируйте ее под классификацию на 8 классов. Создайте подходящую функцию потерь и оптимизатор SGD с `momentum=0.9`.

In [0]:
num_classes = 8

model = models.resnet18(pretrained = True)
model.fc = torch.nn.Linear(512, num_classes)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1.0e-3, momentum=0.9)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

**(10 баллов)** Реализуйте обучение модели, а потом измерьте время инференса на val датасете (`batch_size=32`).

In [17]:
num_epochs = 3

for epoch in tqdm(range(num_epochs)):
    print('Epoch {}/{}:'.format(epoch, num_epochs - 1), flush=True)

    cntLoss = cntAcc = 0.0

    model.train()
    for X, y in train_loader:
        X, y = X.to(device), y.to(device)

        preds = model(X)
        loss = criterion(preds, y)
        predsClass = preds.argmax(dim=1)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cntLoss += loss.item()
        cntAcc += (predsClass == y.data).float().mean()

    print('Train Loss: {:.4f} Acc: {:.4f}'.format(cntLoss, cntAcc), flush=True)

    cntLoss = cntAcc = 0.0

    model.eval()
    for X, y in val_loader:
        X, y = X.to(device), y.to(device)

        preds = model(X)
        loss = criterion(preds, y)
        predsClass = preds.argmax(dim=1)

        cntLoss += loss.item()
        cntAcc += (predsClass == y.data).float().mean()

    print('Validation Loss: {:.4f} Acc: {:.4f}'.format(cntLoss, cntAcc), flush=True)


Epoch 0/2:
Train Loss: 41.3593 Acc: 18.0375
Validation Loss: 11.0486 Acc: 18.1128
Epoch 1/2:
Train Loss: 11.6916 Acc: 27.7625
Validation Loss: 5.0989 Acc: 18.8750
Epoch 2/2:
Train Loss: 6.2709 Acc: 28.8750
Validation Loss: 3.6091 Acc: 19.1562



**(20 баллов)** Установите TensorRT и **torch2trt** (https://github.com/NVIDIA-AI-IOT/torch2trt). Оптимизируйте с помощью torch2trt обученную модель и снова измерьте время инференса.

Попробуйте:
* как режим fp32, так и fp16
* не менее трех разных значений `batch_size`

In [20]:
batchSizeList = [8, 16, 32, 64, 128, 256]
numberOfTimeit = 10
for batch in batchSizeList:
    print('Batch size = {}'.format(batch))

    x = torch.rand((batch, 3, 224, 224)).to(device)  # 32 by default
    xHalf = cp.deepcopy(x.half()).to(device)  # 16 (half)

    model = model.to(device)
    model.eval()

    trt32 = torch2trt(cp.deepcopy(model), [x], fp16_mode=False).to(device)  # fp32 mode
    trt32.eval()

    trt16 = torch2trt(cp.deepcopy(model).half(), [xHalf], fp16_mode=True).to(device)  # fp16 mode (half)
    trt16.eval()

    print('Default model inference time: {}'.format(timeit.timeit(lambda: model(x), number=numberOfTimeit)))
    print('TRT32 model inference time: {}'.format(timeit.timeit(lambda: trt32(x), number=numberOfTimeit)))
    print('TRT16 model inference time: {}'.format(timeit.timeit(lambda: trt16(xHalf), number=numberOfTimeit)))

    print()

Batch size = 8
Default model inference time: 0.023019739000119444
TRT32 model inference time: 0.0022863859999233682
TRT16 model inference time: 0.001527608000287728

Batch size = 16
Default model inference time: 0.025287194000156887
TRT32 model inference time: 0.005774288999873534
TRT16 model inference time: 0.002900592000059987

Batch size = 32
Default model inference time: 0.02341923299991322
TRT32 model inference time: 0.008188585999960196
TRT16 model inference time: 0.007481277999886515

Batch size = 64
Default model inference time: 0.023429252999903838
TRT32 model inference time: 0.0017047520000232907
TRT16 model inference time: 0.0015826559997549339

Batch size = 128
Default model inference time: 0.023086782000063977
TRT32 model inference time: 0.004383216999940487
TRT16 model inference time: 0.0018096949997925549

Batch size = 256
Default model inference time: 0.02852897100001428
TRT32 model inference time: 0.006947355000193056
TRT16 model inference time: 0.006993732999944768



### Бонусная часть (100 баллов)
Нужно переписать функцию `torch2trt()`, чтобы она поддерживала **режим `int8`-инференса**.

Сам TensorRT в режиме `int8` требует **калибровки**, которую нужно выполнять по train датасету. То есть нужно написать класс-калибратор (подкласс `tensorrt.IInt8EntropyCalibrator2`).

За уточнением того, что надо сделать, и за помощью обращаться к:
* https://docs.nvidia.com/deeplearning/sdk/tensorrt-developer-guide/index.html#enable_int8_python
* https://docs.nvidia.com/deeplearning/sdk/tensorrt-api/python_api/index.html
* примеру из TensorRT `samples/python/int8_caffe_mnist`
* преподавателю через почтовый ящик курса или telegram

In [0]:
...